In [ ]:
!pip install --upgrade mlflow
!pip install -U scikit-learn pandas joblib


In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler

# create artifact repository
REPO_ROOT = Path.cwd()
MODELS_DIR = REPO_ROOT / "models/BestModel"
MODELS_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR_RAW = REPO_ROOT  / "data/raw"
DATA_DIR_RAW.mkdir(parents=True, exist_ok=True)
DATA_DIR_PROCESSED = REPO_ROOT / "data/processed"
DATA_DIR_PROCESSED.mkdir(parents=True, exist_ok=True)
print(f"Artifacts will be stored in: {MODELS_DIR}")


# Load dataset
df = pd.read_csv(DATA_DIR_RAW / "california_housing_raw.csv")
#df = housing.frame

# Preprocessing
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.drop("MedHouseVal", axis=1))

# Create processed DataFrame
df_processed = pd.DataFrame(scaled_features, columns=df.drop("MedHouseVal", axis=1).columns)
df_processed["MedHouseVal"] = df["MedHouseVal"]

# Save raw and processed datasets
df.to_csv(DATA_DIR_RAW / "california_housing_raw.csv", index=False)
df_processed.to_csv(DATA_DIR_PROCESSED / "california_housing_processed.csv", index=False)

print("Data loaded and preprocessed successfully.")


X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



mlruns_path = (REPO_ROOT / "mlruns").resolve()
mlruns_path.mkdir(parents=True, exist_ok=True)

# Use a proper file URI
mlflow.set_tracking_uri(mlruns_path.as_uri()) 


# Check and create default experiment if needed
if not mlflow.get_experiment_by_name("Default"):
    mlflow.create_experiment(name="Default")
# ------------------------
# Linear Regression - MLflow Run
# ------------------------

# Check where it's tracking
print("Tracking URI:", mlflow.get_tracking_uri())

mlflow.set_experiment("LogisticRegressionExperiment")
with mlflow.start_run(run_name="LogisticRegressionExperiment") as lr_run:
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_preds = lr_model.predict(X_test)

    input_example = X_test.sample(5)
    predicted = lr_model.predict(input_example)

    # Infer model signature
    signature = infer_signature(input_example, predicted)


    lr_rmse = np.sqrt(mean_squared_error(y_test, lr_preds))  # default squared=True
    lr_r2 = r2_score(y_test, lr_preds)

    print("Logicstic Regression:")
    print(f"RMSE: {lr_rmse:.4f}")
    print(f"R² Score: {lr_r2:.4f}")

    # Log model
    mlflow.sklearn.log_model(lr_model, artifact_path="LogisticRegression", input_example=input_example, signature=signature,  registered_model_name="LogisticRegression")

    print(f"inside run with id : {lr_run.info.run_id}")
    
    # Log metrics
    mlflow.log_metric("rmse", lr_rmse)
    mlflow.log_metric("r2", lr_r2)

# ------------------------
# Decision Tree - MLflow Run
# ------------------------
mlflow.set_experiment("DecisionTreeExperiment")
with mlflow.start_run(run_name="DecisionTreeExperiment") as dt_run:
    dt_model = DecisionTreeRegressor(random_state=42, max_depth=10)
    dt_model.fit(X_train, y_train)
    dt_preds = dt_model.predict(X_test)

    dt_rmse = np.sqrt(mean_squared_error(y_test, dt_preds))
    dt_r2 = r2_score(y_test, dt_preds)

    input_example = X_test.sample(5)
    predicted = dt_model.predict(input_example)

    # Infer model signature
    signature = infer_signature(input_example, predicted)

    print("\n Decision Tree Regressor:")
    print(f"RMSE: {dt_rmse:.4f}")
    print(f"R² Score: {dt_r2:.4f}")

    # Log model
    mlflow.sklearn.log_model(dt_model, artifact_path="DecisionTreeRegression", input_example=input_example, signature=signature,  registered_model_name="DecisionTreeRegression")

    print(f"inside run with id : {dt_run.info.run_id}")

    # Log metrics
    mlflow.log_metric("rmse", dt_rmse)
    mlflow.log_metric("r2", dt_r2)


def dumpModel(model_var_list):
    #model_var_list=list(model)
    # Save model locally
    joblib.dump(model_var_list[0], MODELS_DIR / "betterModel.pkl")



dt_modelparams=[dt_model, dt_rmse, dt_r2, dt_run]
lr_modelparams=[lr_model, lr_rmse, lr_r2, lr_run]


#Choose best model based on rmse and r2 values

if dt_rmse < lr_rmse and dt_r2 > lr_r2:
    better_model = dt_modelparams
elif lr_rmse < dt_rmse and lr_r2 > dt_r2:
    better_model = lr_modelparams
elif dt_r2 > lr_r2:
    better_model = dt_modelparams
else:
    better_model = lr_modelparams   

#dumping model and registering model

dumpModel(better_model)

Artifacts will be stored in: c:\Users\kushl\Documents\MLOpsAssignment\mlops-assignmentOne\notebooks\models\BestModel
Data loaded and preprocessed successfully.
Tracking URI: file:///C:/Users/kushl/Documents/MLOpsAssignment/mlops-assignmentOne/notebooks/mlruns


c:\Users\kushl\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/10 18:30:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logicstic Regression:
RMSE: 0.7456
R² Score: 0.5758


Registered model 'LogisticRegression' already exists. Creating a new version of this model...
Created version '3' of model 'LogisticRegression'.


inside run with id : 572c79f1eced4549946ada600726c081


c:\Users\kushl\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/08/10 18:30:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



 Decision Tree Regressor:
RMSE: 0.6446
R² Score: 0.6829


Registered model 'DecisionTreeRegression' already exists. Creating a new version of this model...
Created version '3' of model 'DecisionTreeRegression'.


inside run with id : 56a3c8958e0e4c06bedbb94fb9f67e5a
